In [ ]:
import tensorflow as tf
import numpy as np
import os
img_size=32
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
data_dict=unpickle("data_batch_1")
data=data_dict[b'data']
the_label=data_dict[b'labels']

test_dict=unpickle("test_batch")
test_data=test_dict[b'data']
test_label=test_dict[b'labels']
print("ok")
#create label to use
    #data
the_label=np.array(the_label)
label_use=np.zeros([10000,10])
for i in range(10000):
    label_use[i][the_label[i]]=1
    
   #test
test_label_buff=np.array(test_label)
test_label=np.zeros([10000,10])
for i in range(10000):
    test_label[i][test_label_buff[i]]=1
    
#create image to use
   
import cv2
import copy
    #data
pix=np.zeros([10000,32,32,3],dtype=np.uint8)
for i in range(10000):
    r=data[i][0:1024]
    g=data[i][1024:2048]
    b=data[i][2048:3072]
    g=g.reshape((32,32))
    b=b.reshape((32,32))
    r=r.reshape((32,32))
    pix[i]=copy.deepcopy(cv2.merge([b,g,r]))
    #test
test_pix=np.zeros([10000,32,32,3],dtype=np.uint8)
for i in range(10000):
    r=test_data[i][0:1024]
    g=test_data[i][1024:2048]
    b=test_data[i][2048:3072]
    g=g.reshape((32,32))
    b=b.reshape((32,32))
    r=r.reshape((32,32))
    test_pix[i]=copy.deepcopy(cv2.merge([b,g,r]))
    
    
img_size=32
batch_size=100
img_channel=3
num_batch=len(the_label)//batch_size
x=tf.placeholder(tf.float32,[None,img_size,img_size,img_channel])
y=tf.placeholder(tf.float32,[None,10])
#w=tf.Variable(tf.zeros([img_size,img_size,3,100]))
#b=tf.Variable(tf.zeros([100]))
def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)
def biase_variable(shape):
    init=tf.constant(0.1,shape=shape)
    return tf.Variable(init)
def weight_init(shape):
    init=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init)
def biase_init(shape):
    init=tf.constant(0.1,shape=shape)
    return tf.Variable(init)
def conv_2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
def max_pool(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
#输入层
x_image=tf.reshape(x,[-1,img_size,img_size,img_channel])
#
W_conv1=weight_variable([5,5,img_channel,30])  # 30 kernels extract feature from one plane
b_conv1=biase_variable([30])

h_conv1=tf.nn.relu(conv_2d(x_image,W_conv1)+b_conv1)
h_pool1=max_pool(h_conv1)

W_conv2=weight_variable([5,5,30,60]) # 60 kernels extract feature from 30 planes
b_conv2=biase_variable([60])

h_conv2=tf.nn.relu(conv_2d(h_pool1,W_conv2)+b_conv2)
h_pool2=max_pool(h_conv2)

W_fc1=weight_variable([(img_size//4)*(img_size//4)*60,1024])
b_fc1=biase_variable([1024])

h_pool2_flat=tf.reshape(h_pool2,[-1,(img_size//4)*(img_size//4)*60])
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

keep_prob=tf.placeholder(tf.float32)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

W_fc2=weight_variable([1024,10])
b_fc2=biase_variable([10])

prediction=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))

train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在布尔变量中
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoth in range(21):
        for batch in range(num_batch):
            batch_x=pix[(batch*batch_size):((batch+1)*batch_size)]
            batch_y=label_use[(batch*batch_size):((batch+1)*batch_size)]
            sess.run(train_step,feed_dict={x:batch_x,y:batch_y,keep_prob:1.0})     
        #acc=sess.run(accuracy,feed_dict={x:test_pix,y: test_label  ,keep_prob:1.0})
        print("iter: "+str(epoth))
print("ok")

In [ ]:
import tensorflow as tf
import numpy as np
import os
img_size=32
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
data_dict=unpickle("data_batch_1")
data=data_dict[b'data']
the_label=data_dict[b'labels']

test_dict=unpickle("test_batch")
test_data=test_dict[b'data']
test_label=test_dict[b'labels']
print("ok")
#create label to use
    #data
the_label=np.array(the_label)
label_use=np.zeros([10000,10])
for i in range(10000):
    label_use[i][the_label[i]]=1
   #test
test_label_buff=np.array(test_label)
test_label=np.zeros([10000,10])
for i in range(10000):
    test_label[i][test_label_buff[i]]=1
    
#create image to use
   
import cv2
import copy
    #data
pix=np.zeros([10000,32,32,3],dtype=np.uint8)
for i in range(10000):
    r=data[i][0:1024]
    g=data[i][1024:2048]
    b=data[i][2048:3072]
    g=g.reshape((32,32))
    b=b.reshape((32,32))
    r=r.reshape((32,32))
    pix[i]=copy.deepcopy(cv2.merge([b,g,r]))
    #test
test_pix=np.zeros([10000,32,32,3],dtype=np.uint8)
for i in range(10000):
    r=test_data[i][0:1024]
    g=test_data[i][1024:2048]
    b=test_data[i][2048:3072]
    g=g.reshape((32,32))
    b=b.reshape((32,32))
    r=r.reshape((32,32))
    test_pix[i]=copy.deepcopy(cv2.merge([b,g,r]))

print("ok")
img_size=32
batch_size=100
img_channel=3
num_batch=len(the_label)//batch_size
x=tf.placeholder(tf.float32,[None,img_size,img_size,img_channel])
y=tf.placeholder(tf.float32,[None,10])
#w=tf.Variable(tf.zeros([img_size,img_size,3,100]))
#b=tf.Variable(tf.zeros([100]))

def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)
def biase_variable(shape):
    init=tf.constant(0.1,shape=shape)
    return tf.Variable(init)

def conv_2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
def max_pool(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#输入层
x_image=tf.reshape(x,[-1,img_size,img_size,img_channel])
#
W_conv1=weight_variable([5,5,img_channel,30])  # 30 kernels extract feature from one plane
b_conv1=biase_variable([30])

h_conv1=tf.nn.relu(conv_2d(x_image,W_conv1)+b_conv1)
h_pool1=max_pool(h_conv1)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoth in range(11):
        for batch in range(num_batch):
            batch_x=pix[(batch*batch_size):((batch+1)*batch_size)]
            batch_y=label_use[(batch*batch_size):((batch+1)*batch_size)]
            pra=sess.run(h_pool1,feed_dict={x:batch_x,y:batch_y})
print("ok")

In [ ]:
img_size=32
batch_size=100
img_channel=3
num_batch=len(the_label)//batch_size
batch=0
batch_x=pix[(batch*batch_size):((batch+1)*batch_size)]


In [ ]:
x=tf.placeholder(tf.float32,shape=[None,img_size,img_size,img_channel])
x_image=tf.reshape(x,[-1,img_size,img_size,img_channel])
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    sess.run(x_image,feed_dict={x:batch_x})

In [ ]:
import tensorflow as tf
import numpy as np
import os
img_size=32
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
data_dict=unpickle("data_batch_1")
data=data_dict[b'data']
the_label=data_dict[b'labels']

test_dict=unpickle("test_batch")
test_data=test_dict[b'data']
test_label=test_dict[b'labels']
print("ok")
#create label to use
 #data
the_label=np.array(the_label)
label_use=np.zeros([10000,10])
for i in range(10000):
    label_use[i][the_label[i]]=1   
   #test
test_label_buff=np.array(test_label)
test_label=np.zeros([10000,10])
for i in range(10000):
    test_label[i][test_label_buff[i]]=1
#create image to use
import cv2
import copy
    #data
pix=np.zeros([10000,img_size*img_size])
for i in range(10000):
    r=data[i][0:1024]
    g=data[i][1024:2048]
    b=data[i][2048:3072]
   # g=g.reshape(img_size*img_size)
  #  b=b.reshape(img_size*img_size)
 #   r=r.reshape(img_size*img_size)
    pix[i]=(r+g+b)/3/255
    #test
test_pix=np.zeros([10000,img_size*img_size])
for i in range(10000):
    r=test_data[i][0:1024]
    g=test_data[i][1024:2048]
    b=test_data[i][2048:3072]
    #g=g.reshape(img_size*img_size)
   # b=b.reshape(img_size*img_size)
  #  r=r.reshape(img_size*img_size)
    test_pix[i]=(r+g+b)/3/255
    #copy.deepcopy(cv2.merge([b,g,r]))
print("ok")

In [ ]:
img_size=32
batch_size=100
img_channel=1
num_batch=len(the_label)//batch_size
x=tf.placeholder(tf.float32,[None,img_size*img_size])
y=tf.placeholder(tf.float32,[None,10])
#w=tf.Variable(tf.zeros([img_size,img_size,3,100]))
#b=tf.Variable(tf.zeros([100]))

def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)
def biase_variable(shape):
    init=tf.constant(0.1,shape=shape)
    return tf.Variable(init)

def conv_2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
def max_pool(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#输入层
x_image=tf.reshape(x,[-1,img_size,img_size,img_channel])
#
W_conv1=weight_variable([5,5,img_channel,30])  # 30 kernels extract feature from one plane
b_conv1=biase_variable([30])

h_conv1=tf.nn.relu(conv_2d(x_image,W_conv1)+b_conv1)
h_pool1=max_pool(h_conv1)
###########################################################
#W_conv2=weight_variable([5,5,30,60]) 
#b_conv2=biase_variable([60])

#h_conv2=tf.nn.relu(conv_2d(h_pool1,W_conv2)+b_conv2)
#h_pool2=max_pool(h_conv2)
###########################################################
W_fc1=weight_variable([(img_size//2)*(img_size//2)*30,1024])
b_fc1=biase_variable([1024])

h_pool2_flat=tf.reshape(h_pool1,[-1,(img_size//2)*(img_size//2)*30])
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

W_fc2=weight_variable([1024,10])
b_fc2=biase_variable([10])
##
prediction=tf.nn.softmax(tf.matmul(h_fc1,W_fc2)+b_fc2)
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))

train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在布尔变量中
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
##
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoth in range(21):
        for batch in range(num_batch):
            if batch<(num_batch-1):
                batch_x=pix[(batch*batch_size):((batch+1)*batch_size)]
                batch_y=label_use[(batch*batch_size):((batch+1)*batch_size)]
                sess.run(train_step,feed_dict={x:batch_x,y:batch_y})
        acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
        print("iter: "+str(epoth)+"acc  "+str(acc))  
print("ok")

In [ ]:
batch_x[2]

In [ ]:
import tensorflow as tf
import numpy as np
import os
img_size=32
#os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
    
train_data=[]
train_label=[]
for i in range(5):
    data_dict=unpickle("data_batch_"+str(i+1))
    train_data=np.append(train_data,data_dict[b'data'])
    the_label=np.append(the_label,data_dict[b'labels'])


test_dict=unpickle("test_batch")
test_data=test_dict[b'data']
test_label=test_dict[b'labels']
print("ok")
#create label to use
 #data
the_label=np.array(the_label)
label_use=np.zeros([10000,10])
for i in range(10000):
    label_use[i][the_label[i]]=1   
   #test
test_label_buff=np.array(test_label)
test_label=np.zeros([10000,10])
for i in range(10000):
    test_label[i][test_label_buff[i]]=1
#create image to use
#import cv2
import copy
    #data
pix=np.zeros([10000,img_size*img_size])
for i in range(10000):
    r=data[i][0:1024]/3
    g=data[i][1024:2048]/3
    b=data[i][2048:3072]/3
    pix[i]=(r+g+b)
   
#test
test_pix=np.zeros([10000,img_size*img_size])
for i in range(10000):
    r=test_data[i][0:1024]/3
    g=test_data[i][1024:2048]/3
    b=test_data[i][2048:3072]/3
    #g=g.reshape(img_size*img_size)
   # b=b.reshape(img_size*img_size)
  #  r=r.reshape(img_size*img_size)
    test_pix[i]=(r+g+b)
np.save("test_data.npy",test_pix)
np.save("test_label.npy",test_label)
np.save("train_data.npy",pix)
np.save("train_label.npy",label_use)
    #copy.deepcopy(cv2.merge([b,g,r]))
print("ok")

In [18]:
import numpy as np
import copy
import cv2
img_size=32
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

test_dict=unpickle("test_batch")
test_data_buff=test_dict[b'data']
test_label_buff=test_dict[b'labels']

for i in range(5):
    data_dict=unpickle("data_batch_"+str(i+1))
    if i==0:
        train_data_buff=data_dict[b'data']
        train_label_buff=data_dict[b'labels']
    else:
        train_data_buff=np.concatenate((train_data_buff,data_dict[b'data']),axis=0)
        train_label_buff=np.concatenate((train_label_buff,data_dict[b'labels']),axis=0)
print("ok")

train_data=np.zeros([50000,img_size,img_size,3])
train_label=np.zeros([50000,10])
test_data=np.zeros([10000,img_size,img_size,3])
test_label=np.zeros([10000,10])
for i in range(50000):
    #train data
    r=train_data_buff[i][0:1024]
    g=train_data_buff[i][1024:2048]
    b=train_data_buff[i][2048:3072]
    g=g.reshape(img_size,img_size)
    b=b.reshape(img_size,img_size)
    r=r.reshape(img_size,img_size)
    train_data[i]=copy.deepcopy(cv2.merge([b,g,r]))
    #test data
    train_label[i][train_label_buff[i]]=1
    #
    if i<10000:
        r=test_data_buff[i][0:1024]
        g=test_data_buff[i][1024:2048]
        b=test_data_buff[i][2048:3072]
        g=g.reshape((32,32))
        b=b.reshape((32,32))
        r=r.reshape((32,32))
        test_data[i]=copy.deepcopy(cv2.merge([b,g,r]))
        test_label[i][test_label_buff[i]]=1
print("ok")
np.save("c_test_data.npy",test_data)
np.save("c_test_label.npy",test_label)
np.save("c_train_data.npy",train_data)
np.save("c_train_label.npy",train_label)

ok
ok


In [ ]:
train_data_buff.shape

In [ ]:
a=np.array([[1,2,3],[2,3,4]])

In [ ]:
b=np.array([[1,2,3],[2,3,4]])

In [ ]:
a=np.append(a,b)

In [4]:
train_label_buff[0:100]

array([6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, 9, 9, 3, 2, 6, 4, 3,
       6, 6, 2, 6, 3, 5, 4, 0, 0, 9, 1, 3, 4, 0, 3, 7, 3, 3, 5, 2, 2, 7,
       1, 1, 1, 2, 2, 0, 9, 5, 7, 9, 2, 2, 5, 2, 4, 3, 1, 1, 8, 2, 1, 1,
       4, 9, 7, 8, 5, 9, 6, 7, 3, 1, 9, 0, 3, 1, 3, 5, 4, 5, 7, 7, 4, 7,
       9, 4, 2, 3, 8, 0, 1, 6, 1, 1, 4, 1])

In [22]:
test_label[0:10]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [8]:
np.save("train_data.npy",train_data)

np.save("train_label.npy",train_label)

In [19]:
test_label[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [21]:
test_dict[b'filenames']

[b'domestic_cat_s_000907.png',
 b'hydrofoil_s_000078.png',
 b'sea_boat_s_001456.png',
 b'jetliner_s_001705.png',
 b'green_frog_s_001658.png',
 b'crapaud_s_002124.png',
 b'shooting_brake_s_000973.png',
 b'green_frog_s_000634.png',
 b'tabby_cat_s_001397.png',
 b'wagon_s_002806.png',
 b'plane_s_000026.png',
 b'dustcart_s_000045.png',
 b'toy_spaniel_s_001592.png',
 b'lippizan_s_000752.png',
 b'truck_s_000149.png',
 b'dredger_s_001437.png',
 b'puppy_s_001937.png',
 b'cow_pony_s_000038.png',
 b'boat_s_001450.png',
 b'rana_clamitans_s_000570.png',
 b'tennessee_walker_s_000353.png',
 b'stealth_fighter_s_000354.png',
 b'wapiti_s_001434.png',
 b'aerial_ladder_truck_s_001218.png',
 b'puppy_s_001691.png',
 b'gamecock_s_000228.png',
 b'elk_s_000015.png',
 b'dive_bomber_s_000184.png',
 b'tipper_truck_s_000866.png',
 b'pickerel_frog_s_001165.png',
 b'bufo_boreas_s_000096.png',
 b'pekingese_s_000608.png',
 b'deer_s_001275.png',
 b'puppy_s_000644.png',
 b'transporter_s_000374.png',
 b'struthio_camelus_